# CIFAR 10 Dataset Image Classifier

## Importing required libraries

In [1]:
#!pip install --upgrade keras-cv tensorflow
#!pip install --upgrade keras
#!pip install keras_tuner
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import keras
import keras_tuner as kt


## Loading Dataset

[Cifar-10 Dataset](https://keras.io/api/datasets/cifar10/)

In [2]:
cifar10 = tf.keras.datasets.cifar10.load_data()
(X_train_full, y_train_full), (X_test, y_test) = cifar10


### Converting images from RGB to Grayscale
By converting the images to grayscale we reduce their dimension which helps in reducing the computations computed while training the model.

In [3]:
X_train_full = tf.image.rgb_to_grayscale(X_train_full)
X_test = tf.image.rgb_to_grayscale(X_test)

In [4]:
X_train_full.shape

TensorShape([50000, 32, 32, 1])

### Splitting training data into train and split.


In [6]:
X_train, y_train = X_train_full[:-10000], y_train_full[:-10000]
X_valid, y_valid = X_train_full[-10000:], y_train_full[-10000:]

In [7]:
print(X_train[1][0][1])

tf.Tensor([133], shape=(1,), dtype=uint8)


### Encoding Class names

In [8]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
class_names[int(y_train[5])]


<ipython-input-8-f94cb98b12e9>:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  class_names[int(y_train[5])]


'automobile'

## Defining Neural network's number of layers, number of neurons, activation functions.

In [13]:



model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = [32, 32, 1]),
    tf.keras.layers.Dense(800, kernel_initializer = 'random_normal',bias_initializer = 'zeros' ,activation = "relu"),
    tf.keras.layers.Dense(700, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(400, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(10, activation = "softmax")
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_21 (Dense)            (None, 800)               820000    
                                                                 
 dense_22 (Dense)            (None, 700)               560700    
                                                                 
 dense_23 (Dense)            (None, 600)               420600    
                                                                 
 dense_24 (Dense)            (None, 600)               360600    
                                                                 
 dense_25 (Dense)            (None, 600)               360600    
                                                                 
 dense_26 (Dense)            (None, 400)              

### Defining model's optimizer, loss, metrics

In [14]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = "accuracy")

### Defining callbacks and fitting the model
The early stopping callback ensures that the model stops fitting data if it hasn't observed significant change in loss over the given 'patience' number of epochs.

In [17]:
checkpoints = tf.keras.callbacks.ModelCheckpoint('./checkpoint.model.keras')
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience = 10,
                                                     restore_best_weights = True)
history = model.fit(X_train, y_train, epochs = 30, validation_data = (X_valid, y_valid), callbacks=[checkpoints, early_stopping_cb])

Epoch 1/30
1250/1250 [==============================] - 5s 4ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 2/30
1250/1250 [==============================] - 6s 5ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 3/30
1250/1250 [==============================] - 5s 4ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 4/30
1250/1250 [==============================] - 5s 4ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 5/30
1250/1250 [==============================] - 6s 5ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 6/30
1250/1250 [==============================] - 5s 4ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 7/30
1250/1250 [==============================] - 6s 5ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 8/30
1250/1250 [============

### Evaluating the model

In [19]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1000


[nan, 0.10000000149011612]

## Tuning Hyperparameters

Number of Hidden layers, Number of neurons in hiddens layers, Learning rate of optimizer, Choice of optimizer are conisdered as hyperparameters below. We then perform RandomSearch on the hyperparemeters with their ranges defined below.

In [ ]:
def build_model(hp):
  n_hidden = hp.Int("n_hidden", min_value = 0, max_value = 10, default = 2)
  n_neurons = hp.Int("n_neurons", min_value = 10, max_value = 500, default = 3)
  learning_rate = hp.Float("learning_rate", min_value = 1e-4, max_value = 1e-2, sampling = "log")
  optimizer = hp.Choice("optimizer", values = ["sgd", "adam"])
  if optimizer == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
  else:
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape = [32,32,1]))
  for i in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_neurons, activation = "relu"))
  model.add(tf.keras.layers.Dense(10, activation = "softmax"))
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
  return model



In [ ]:
random_search_tuner = kt.RandomSearch(build_model,
                                  objective = "accuracy",
                                  max_trials = 100,
                                  seed = 42,
                                  directory = "cifarproject",
                                  project_name = "random_search_runs")
random_search_tuner.search(X_train, y_train, epochs = 5, validation_data = (X_valid, y_valid))

Trial 100 Complete [00h 00m 42s]
accuracy: 0.2178249955177307

Best accuracy So Far: 0.3695499897003174
Total elapsed time: 01h 00m 58s


### Let's retrieve top 5 trial models' hyperparameters

In [ ]:
top5_trials = random_search_tuner.oracle.get_best_trials(num_trials = 5)
for _ in top5_trials:
  print(_.summary())

Trial 019 summary
Hyperparameters:
n_hidden: 7
n_neurons: 464
learning_rate: 0.0010712131812413617
optimizer: sgd
Score: 0.3695499897003174
None
Trial 017 summary
Hyperparameters:
n_hidden: 10
n_neurons: 416
learning_rate: 0.0009172617171127613
optimizer: sgd
Score: 0.3644999861717224
None
Trial 060 summary
Hyperparameters:
n_hidden: 7
n_neurons: 365
learning_rate: 0.00011631756169915276
optimizer: adam
Score: 0.35534998774528503
None
Trial 034 summary
Hyperparameters:
n_hidden: 10
n_neurons: 452
learning_rate: 0.00019430304760198347
optimizer: adam
Score: 0.3549000024795532
None
Trial 032 summary
Hyperparameters:
n_hidden: 8
n_neurons: 393
learning_rate: 0.0005070370705614628
optimizer: sgd
Score: 0.35397499799728394
None


### Defining Neural Network
Based on the top 5 trial models' hyperparameters lets define a neural network with more number of layers and neurons.

In [9]:
best_trial = tf.keras.Sequential()
best_trial.add(tf.keras.layers.Flatten(input_shape = [32,32,1]))
for _ in range(20):
  best_trial.add(tf.keras.layers.Dense(950, activation = "relu"))
best_trial.add(tf.keras.layers.Dense(10, activation = "softmax"))

best_trial.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 950)               973750    
                                                                 
 dense_1 (Dense)             (None, 950)               903450    
                                                                 
 dense_2 (Dense)             (None, 950)               903450    
                                                                 
 dense_3 (Dense)             (None, 950)               903450    
                                                                 
 dense_4 (Dense)             (None, 950)               903450    
                                                                 
 dense_5 (Dense)             (None, 950)               9

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)
best_trial.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

In [11]:
checkpoints_b = tf.keras.callbacks.ModelCheckpoint('./checkpoint_b.model.keras')
early_stopping_cb_b = tf.keras.callbacks.EarlyStopping(patience = 10,
                                                     restore_best_weights = True)
history_b = best_trial.fit(X_train, y_train, epochs = 40, validation_data = (X_valid, y_valid), callbacks=[early_stopping_cb_b, checkpoints_b])

Epoch 1/40
1250/1250 [==============================] - 21s 11ms/step - loss: 2.1391 - accuracy: 0.1784 - val_loss: 2.0636 - val_accuracy: 0.2058
Epoch 2/40
1250/1250 [==============================] - 13s 10ms/step - loss: 2.0711 - accuracy: 0.2037 - val_loss: 2.0804 - val_accuracy: 0.2176
Epoch 3/40
1250/1250 [==============================] - 14s 11ms/step - loss: 2.0106 - accuracy: 0.2396 - val_loss: 1.9697 - val_accuracy: 0.2560
Epoch 4/40
1250/1250 [==============================] - 13s 11ms/step - loss: 1.9694 - accuracy: 0.2604 - val_loss: 1.9568 - val_accuracy: 0.2585
Epoch 5/40
1250/1250 [==============================] - 13s 10ms/step - loss: 1.9446 - accuracy: 0.2709 - val_loss: 1.9129 - val_accuracy: 0.2735
Epoch 6/40
1250/1250 [==============================] - 13s 11ms/step - loss: 1.9216 - accuracy: 0.2803 - val_loss: 1.9292 - val_accuracy: 0.2766
Epoch 7/40
1250/1250 [==============================] - 13s 11ms/step - loss: 1.8799 - accuracy: 0.2964 - val_loss: 1.9437 -

### Test accuracy of our final Neural Network

In [12]:
best_trial.evaluate(X_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.7512 - accuracy: 0.4005


[1.7512485980987549, 0.40049999952316284]